<a href="https://colab.research.google.com/github/sruane1/testing_misc/blob/main/EIN_PreProcessing_PDF_OCR%20Revised%2012152022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert PDF to OCR Searchable PDF for EIN Scanning & Retrieval

## Lib imports

In [13]:
! apt install tesseract-ocr
! apt install libtesseract-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [14]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!apt install ghostscript

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.26~dfsg+0-0ubuntu0.18.04.17).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


Prior version of ocrmypdf required due to tesseract version error in latest

In [30]:
!pip install ocrmypdf==13.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 31.2 MB/s 
     |████████████████████████████████| 46 kB 4.7 MB/s 
     |████████████████████████████████| 5.6 MB 66.0 MB/s 
     |████████████████████████████████| 86 kB 7.4 MB/s 
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20221105
    Uninstalling pdfminer.six-20221105:
      Successfully uninstalled pdfminer.six-20221105
  Attempting uninstall: ocrmypdf
    Found existing installation: ocrmypdf 3.2.1
    Uninstalling ocrmypdf-3.2.1:
      Successfully uninstalled ocrmypdf-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
p

In [1]:
from ctypes.util import find_library
find_library("gs")

'libgs.so.9'

In [2]:
#optional imports to make a pretty list
import pandas as pd
%load_ext google.colab.data_table

In [3]:
!mkdir input_pdfs
!mkdir output_pdfs
!mkdir pdfs_processed

mkdir: cannot create directory ‘input_pdfs’: File exists
mkdir: cannot create directory ‘output_pdfs’: File exists
mkdir: cannot create directory ‘pdfs_processed’: File exists


## OCR Execution

<==== click on the Folder logo.  
Drag and drop you files in the "input_papers" folder.  
The output files will be located in the "output_papers" folder.  
(If the folders are not visible, refresh the explorer with the refresh button)

In [4]:
import os
file_list = []

input_folder_path = "/content/input_pdfs"
output_folder_path = "/content/output_pdfs"

for filename in os.listdir(input_folder_path):
    if filename.endswith("pdf"): 
      file_list.append(filename)

print("The following files will be converted : ") 
pd.DataFrame(file_list, columns=["Filename"])

The following files will be converted : 


,Filename
0,sfr_tst_animal_league.pdf
1,(nosearch)IRS-tax-exempt-status-affirmation-le...
2,test_ein_letter.pdf
3,sfr_tst.pdf
4,IRS Determination Letter-Global Resource for A...
5,IRS-tax-exempt-status-affirmation-letter.pdf


Warning : several minutes per file, please check the above list before proceeding.

In [5]:
for filename in file_list:
  print("Converting : " + filename)
  input_filename = input_folder_path+'/'+filename
  output_filename = output_folder_path+'/'+filename
  !ocrmypdf --force-ocr "{input_filename}" "{output_filename}"

Converting : sfr_tst_animal_league.pdf
Scanning contents: 100% 1/1 [00:00<00:00, 51.70page/s]
    1 page already has text! - rasterizing text and running OCR anyway
OCR: 100% 1.0/1.0 [00:09<00:00,  9.47s/page]
Postprocessing...
PDF/A conversion: 100% 1/1 [00:00<00:00,  4.83page/s]
Some input metadata could not be copied because it is not permitted in PDF/A. You may wish to examine the output PDF's XMP metadata.
Recompressing JPEGs: 0image [00:00, ?image/s]
Deflating JPEGs: 100% 1/1 [00:00<00:00, 34.03image/s]
JBIG2: 0item [00:00, ?item/s]
Optimize ratio: 1.32 savings: 24.3%
Output file is a PDF/A-2B (as expected)
Converting : (nosearch)IRS-tax-exempt-status-affirmation-letter.pdf
Scanning contents: 100% 2/2 [00:00<00:00, 216.33page/s]
Start processing 2 pages concurrently
    1 [tesseract] unsure about page orientation
OCR: 100% 2.0/2.0 [00:04<00:00,  2.21s/page]
Postprocessing...
PDF/A conversion: 100% 2/2 [00:00<00:00,  3.60page/s]
Some input metadata could not be copied because it i

In [6]:
!pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from pdfminer import high_level
import os
import re
import shutil


In [8]:
for filename in file_list:
    print("Converting : " + filename)
    input_filename = input_folder_path+'/'+filename
    output_filename = output_folder_path+'/'+filename
    output_pdf_processed = '/content/pdfs_processed'
    f = os.path.join(output_folder_path, filename)
    if os.path.isfile(f):
        local_pdf_filename = f
        pages = [0]
        extracted_text = high_level.extract_text(local_pdf_filename, "", pages)
        lines = extracted_text.split("\n")
        ein_find = re.compile(r'^\w+\d?-\w+')

        for line in lines:
            if ein_find.search(line):
                if len(line.strip()) == 10:
                    ein = line.strip()
                    g = os.path.join(output_folder_path+'/', ein + '_' + filename)
                    end_file = ein + '_' + filename
                    shutil.copyfile(output_folder_path+'/' + filename, output_pdf_processed+'/' + end_file)

Converting : sfr_tst_animal_league.pdf
Converting : (nosearch)IRS-tax-exempt-status-affirmation-letter.pdf
Converting : test_ein_letter.pdf
Converting : sfr_tst.pdf
Converting : IRS Determination Letter-Global Resource for Advancing Cancer Education-11.pdf
Converting : IRS-tax-exempt-status-affirmation-letter.pdf
